In [ ]:
# Notebook trying out Liebling's method for separating periodic structures from background and non-periodic
from analyze_dataset import *

In [ ]:
basePath = '/Users/jonny/Movies/Reference_brightfield_video_sequences/'
frameRanges = [(0, 500)]
frameRanges = None

# Pasted from analyze_dataset
imageRangeList = frameRanges
annotationList = []
# Defaults
downsampling = 1
numSamplesPerPeriod = 60
earlyAnnotationTruncation = -1
source = 'Brightfield - Prosilica'
periodRange = np.arange(23, 37, 0.1)
plotAllPeriods=False
alpha=10
cropRect=None
cropFollowsZ=None
maxOffsetToConsider=None
applyDriftCorrection=True
annotationTimeShifts=None
interpolationDistanceBetweenSequences=0
rollingAverage=False
sourceTimestampKey='time_received'
fluorTimestampKey='time_exposed'
filenameFormat=None
# Overrides
periodRange = np.arange(35, 50, 0.1)
numSamplesPerPeriod = 80
source='Nov 2011 Prosilica renumbered'
applyDriftCorrection=False
downsampling=2
interpolationDistanceBetweenSequences=4
sourceTimestampKey='time_received'
fluorTimestampKey='time_received'


# Load brightfield images for analysis
if (filenameFormat is None):
    pathFormat = '%s/'+source+'/%06d.tif'
else:
    pathFormat = '%s/'+source+'/'+filenameFormat
if (imageRangeList is None):
    (images, averagePeriod) = LoadAllImages(basePath+'/'+source, downsampleFactor=downsampling, periodRange=periodRange, plotAllPeriods=plotAllPeriods, cropRect=cropRect, cropFollowsZ=cropFollowsZ, timestampKey=sourceTimestampKey)
else:
    assert(len(imageRangeList) > 0)
    (firstImage, lastImage) = imageRangeList[0]
    (images, averagePeriod) = LoadImages(basePath, pathFormat, firstImage, lastImage - firstImage + 1, downsampleFactor=downsampling, periodRange=periodRange, plotAllPeriods=plotAllPeriods, cropRect=cropRect, cropFollowsZ=cropFollowsZ, timestampKey=sourceTimestampKey)
    for i in range(1, len(imageRangeList)):
        (firstImage, lastImage) = imageRangeList[i]
        lastIndex = images[-1].frameIndex
        (im2, dummy) = LoadImages(basePath, pathFormat, firstImage, lastImage - firstImage + 1, downsampleFactor=downsampling, periodRange=periodRange, plotAllPeriods=plotAllPeriods, frameIndexStart=lastIndex+1, cropRect=cropRect, cropFollowsZ=cropFollowsZ, timestampKey=sourceTimestampKey)
        images = np.append(images, im2)
        del im2

# Temp: apply rolling 4-point average for the images (to deal with noise on the raw compressive images)
if (rollingAverage):
    for i in range(len(images)-3):
        images[i].image = (images[i].image + images[i+1].image + images[i+2].image + images[i+3].image) / 4.0
        images[i].timestamp = (images[i].timestamp + images[i+1].timestamp + images[i+2].timestamp + images[i+3].timestamp) / 4.0



# Looking at the scores when determining the period, there is a clear drop-off in score at higher values of period
# I suspect this is because if time-sequential frames are adjacent in the phase-wrapped version
# then the overall penalties are much lower (adjacent frames are more similar than frames one beat apart, due to RBCs etc)
# This means that our candidate period bracket cannot be too generous.
# For the "slow heart belly scan" dataset, a range of (20, 50) finds 50-ish for sequence 10, instead of 30ish.

# Split into two-period chunks for phase alignment.
# (doing two periods because Liebling's approach typically requires multiple periods in one chunk in order to be able to determine the period.
# This could perhaps be improved using my method for determining the period in the first place.
(imageSections, sectionPeriods) = SplitIntoSections(images, averagePeriod, periodRange=periodRange, plotAllPeriods=plotAllPeriods, alpha=alpha)

# Resample each image section into exactly the same number of frames (as per Liebling's papers)
print('period range', np.min(sectionPeriods), 'to', np.max(sectionPeriods))
print (len(imageSections), 'image sections')
resampledImageSections = ResampleUniformly(imageSections, sectionPeriods, numSamplesPerPeriod)

# Attempt to free up memory by freeing the raw images - we don't need them any more
del images
del imageSections

# Optional: crude attempt to correct for xy drift in the brightfield images
# (which will occur due to focus correction as well as due to actual fish motion)
if applyDriftCorrection:
    driftInset = 10     # TODO: Effectively this is the maximum drift we can handle. Needs to be coded properly!
    sequenceDrifts = CorrectForDrift(resampledImageSections, numSamplesPerPeriod, inset=driftInset)
else:
    driftInset = 0
    sequenceDrifts = [(0, 0)] * len(resampledImageSections)

# Determine the relative shifts between pairs of image sections
shifts = GetShifts(resampledImageSections, sectionPeriods, sequenceDrifts, driftInset, numSamplesPerPeriod, maxOffsetToConsider)
print(shifts)
# Turn these relative shifts into one self-consistent global set of absolute shifts
(globalShiftSolution, adjustedShifts, adjacentSolution, res, res2) = MakeShiftsSelfConsistent(shifts, len(resampledImageSections), numSamplesPerPeriod)

#print (globalShiftSolution - adjacentSolution)

if False:
    for i in range(len(shifts)):
        # Look at how each measured shift compares with the solution we obtained
        (i, j, sh, sc) = shifts[i]
        print(i, j, sh, sc, (globalShiftSolution[j]-globalShiftSolution[i])%numSamplesPerPeriod, (adjacentSolution[j]-adjacentSolution[i])%numSamplesPerPeriod)

# Look at how the global solution differs from the adjacent solution
# (bearing in mind that we don't necessarily know which one to trust!)
# Note that it would be informative to work with a long video that truly is
# just focused on a single plane. That would help test how much the shifts
# will naturally wander due to random variation.
# sqrt(n) random walk would suggest wandering 17 in 300, which is about what I see, in fact.
# Another potentially interesting thing to do would be to do a z scan and then retreat
# back to where we started (if this can be done smoothly), and see how much the phase has drifted.
# Yet another would be to run the same analysis but with different resampling,
# and see how similar the results are.
#plt.plot(((adjacentSolution - globalShiftSolution + numSamplesPerPeriod/2.0) % numSamplesPerPeriod) - numSamplesPerPeriod/2.0)

#print (adjacentSolution[50:80] % numSamplesPerPeriod)
#print (globalShiftSolution[50:80] % numSamplesPerPeriod)
#print (((adjacentSolution - globalShiftSolution)[50:80]) % numSamplesPerPeriod)


# Define phases for resampled image sections
# These give our definitive known time/phase mapping
(knownTimes, knownPhases) = DefinePhaseForSequence(resampledImageSections, globalShiftSolution, numSamplesPerPeriod, plotIt = True, interpolationDistanceBetweenSequences=interpolationDistanceBetweenSequences)

In [ ]:
# Now we apply the shifts to each image sequence, so that we have a series of sequences that should be mutually synchronized
shiftedSequences = []
for i in range(len(resampledImageSections)):
    seq = resampledImageSections[i]
    shifted = np.roll(np.array(seq), int(np.round(-globalShiftSolution[i])))
    shiftedSequences.append(shifted)


In [ ]:
# To check we have synchronized correctly, output a set of folders with the images from the first few sequences
if False:
    for i in range(4):
        SaveImagesToFolder(shiftedSequences[i], '/Users/jonny/Movies/temp/%d'%i)

# Visual examination suggests it does an OK but not perfect job.
# This may be due to the use of two heartbeats, or due due to genuine variation in how a single beat looks.
# I haven't investigated any further for now.
# (Of course, Liebling's nonuniform registration deals with some of these issues)


In [ ]:
# Reformat the data into a single huge matrix (risk of filling memory...?)
bigMatrix = np.zeros((len(shiftedSequences), len(shiftedSequences[0]), shiftedSequences[0][0].image.shape[0], shiftedSequences[0][0].image.shape[1]))
for i in range(len(shiftedSequences)):
    for j in range(len(shiftedSequences[i])):
        bigMatrix[i,j,:,:] = shiftedSequences[i][j].image

In [ ]:
# Now try categorizing. 
# First identify the median, separately for each pixel and each phase
medians = np.zeros((bigMatrix.shape[1], bigMatrix.shape[2], bigMatrix.shape[3]))
for i in range(bigMatrix.shape[1]): # Iterate over phase
    # For each phase, calculate the median for each pixel
    # (which is presumably some sort of measure of typical appearance).
    # (I don't know why they chose median in particular. Seems like an odd choice to me)
    medians[i] = np.median(bigMatrix[:,i,:,:], axis=0)

In [ ]:
# Liebling defines stationary structures as the mean of this value over all phases
stationaryStructures = np.mean(medians, axis=0)
#stationaryStructures[22,99:] = 255
plt.imsave('/Users/jonny/Movies/temp/stationary/0.tif', stationaryStructures, cmap='gray', vmin=0, vmax=255)
% matplotlib inline
plt.plot(medians[:,22,99])
plt.show()
print(np.mean(medians[:,22,99]))
print(stationaryStructures[22,99])

In [ ]:
# Periodic structures are then defined as medians minum stationary background
periodicStructures = np.zeros(medians.shape)
for i in range(bigMatrix.shape[1]): # Iterate over phase
    periodicStructures[i] = medians[i] - stationaryStructures
    plt.imsave('/Users/jonny/Movies/liebling_separation/medians/%d.tif'%i, medians[i], cmap='gray', vmin=0, vmax=255)
    plt.imsave('/Users/jonny/Movies/liebling_separation/periodic/%d.tif'%i, periodicStructures[i], cmap='gray', vmin=0, vmax=255)
    plt.imsave('/Users/jonny/Movies/liebling_separation/sum/%d.tif'%i, periodicStructures[i]+stationaryStructures, cmap='gray', vmin=0, vmax=255)

In [ ]:
# Alternative: process images by binning of realtime phase stamps

if False:
    # Actual overnight dataset (with focus correction of course)
    realtimePath = '/Users/jonny/Movies/Reference_brightfield_video_sequences/9 feb stack 1/Emulated camera'
    (realtimeImages, dummy) = LoadAllImages(realtimePath, downsampleFactor=1, periodRange=None, plotAllPeriods=False)
elif False:
    # A PIV dataset where I was dwelling on a single plane
    # (which will make things more consistent in terms of how the heart looks in brightfield)
    realtimePath = '/Users/jonny/Movies/Reference_brightfield_video_sequences/30 jun 16.03 single plane/Emulated camera'
    (realtimeImages, dummy) = LoadImages(realtimePath, '%s/%06d.tif', 0, 20000, downsampleFactor=downsampling, periodRange=periodRange, plotAllPeriods=plotAllPeriods, cropRect=cropRect, cropFollowsZ=cropFollowsZ, timestampKey=sourceTimestampKey)
elif True:
    # A possible Casper dataset
    realtimePath = '/Users/jonny/Movies/Reference_brightfield_video_sequences/11 jul 2013 maybe casper/Emulated camera'
    (realtimeImages, dummy) = LoadAllImages(realtimePath, downsampleFactor=1, periodRange=None, plotAllPeriods=False)

In [ ]:
# Reformat the data into a single huge matrix (risk of filling memory...?),
# correct for drift while we're at it,
# and identify the (realtime) phases for each image
bigMatrix2 = np.zeros((len(realtimeImages), realtimeImages[0].image.shape[0], realtimeImages[0].image.shape[1]))
phaseArray = np.zeros((len(realtimeImages)))
drifts = np.zeros((len(realtimeImages), 2))
for i in range(len(realtimeImages)):
    driftOffset = realtimeImages[i].plist['drift_offset']
    # Tedious parsing of drift string
    temp = (driftOffset.split("{")[1]).split("}")[0]
    coords = temp.split(",")
    driftX = int(coords[0])
    driftY = int(coords[1])
    drifts[i,0] = driftX
    drifts[i,1] = driftY
    # IMPORTANT: the reason I have different signs on my drift corrections for each axis
    # is because these images have been through EmulatedCamera with horizontal flip enabled!
    # So the sign on the x axis needs to be reversed in order to make sense with the images
    # that were actually saved
    if False:
        # No drift correction
        bigMatrix2[i,:,:] = realtimeImages[i].image
    elif False:
        # Attempted drift correction
        bigMatrix2[i,:,:] = np.roll(np.roll(realtimeImages[i].image, driftX, axis=1), -driftY, axis=0)
    else:
        # Shift by half the drift quantity.
        # Even though this has no basis in reality, it seems to actually give fairly ok results for
        # the PIV dataset, for some reason!
        bigMatrix2[i,:,:] = np.roll(np.roll(realtimeImages[i].image, int(driftX/2), axis=1), -int(driftY/2), axis=0)

    refPos = realtimeImages[i].plist['sync_info']['ref_pos_without_padding']
    refPeriod = realtimeImages[i].plist['sync_info']['reference_period']
    if False:#((refPos > 10) and (refPos < 10.2)):
        plt.imsave('/Users/jonny/Movies/temp/temp/rolled_%d.tif'%i, bigMatrix2[i], cmap='gray', vmin=0, vmax=255)
    if (refPeriod > 0):
        phaseArray[i] = refPos / refPeriod
    else:
        phaseArray[i] = -1    
        
plt.plot(drifts[:,0])
plt.plot(drifts[:,1])
plt.show()

In [ ]:
# Divide the phases into a series of bins, and calculate medians for each one
numPhaseBins = 40
rtMedians = np.zeros((numPhaseBins, realtimeImages[0].image.shape[0], realtimeImages[0].image.shape[1]))
for i in range(numPhaseBins):
    minPhase = i / numPhaseBins
    maxPhase = (i+1) / numPhaseBins
    # Identify frames belonging to this phase bin
    # Note that the > condition skips frames with a phase of 0
    matches = np.where((phaseArray > minPhase) & (phaseArray <= maxPhase))
    # Calculate the median for each pixel, over these phase bins
    if False:
        # I am not totally confident that this is correct
        # due to the way I use matches as an index into bigMatrix2
        # I am alarmed by the fact that matrixForThisPhase comes out as a 4D matrix!
        matrixForThisPhase = bigMatrix2[matches, :, :]
    else:
        matrixForThisPhase = bigMatrix2[matches, :, :]
        
    rtMedians[i] = np.median(matrixForThisPhase[0,:,:,:], axis=0)
    #print(i, matrixForThisPhase.shape, rtMedians.shape)

    plt.imsave('/Users/jonny/Movies/temp/mediansRT/%d.tif'%i, rtMedians[i], cmap='gray', vmin=0, vmax=255)

In [ ]:
# Liebling defines stationary structures as the mean of this value over all phases
rtStationaryStructures = np.mean(rtMedians, axis=0)
#stationaryStructures[22,99:] = 255
plt.imsave('/Users/jonny/Movies/temp/stationaryRT/0.tif', rtStationaryStructures, cmap='gray', vmin=0, vmax=255)

# Periodic structures are then defined as medians minum stationary background
rtPeriodicStructures = np.zeros(rtMedians.shape)
for i in range(numPhaseBins): # Iterate over phase
    rtPeriodicStructures[i] = rtMedians[i] - rtStationaryStructures + 128
    plt.imsave('/Users/jonny/Movies/temp/periodicRT/%d.tif'%i, rtPeriodicStructures[i], cmap='gray', vmin=0, vmax=255)

In [ ]:
# ****** When I run this I simply don't get as satisfying results as Liebling.
# I wonder if it is that I don't have a high enough magnification, and I have a large DoF.
# My changes are actually just a few pixels,
# and I think that perhaps that's preventing the algorithm from finding a
# satisfactory background on which the changes can be observed?
# It is also interesting to note that I am not having full success in eliminating the RBCs from the median images.
# This may be the larger DoF, meaning I have more overall clutter than him.
#
# I think the fact that the background does look like one particular phase in the heart is probably
# my problem here. If there was no heart visible at all, then I suspect it would all work a lot better.
# I think it helps them a lot that they have a very uniform background around the heart,
# (and again, their smaller DoF)
#
# I also haven't done their thing of not actually using all the beats, but just some of them
# 
# Either way, immediate conclusion is that it is not trivial to apply their technique to my raw videos!
#
# For publication, I could consider taking casper video on an upright microscope at various timepoints,
# and seeing if that looks better and processes better.
#